Bước 1: Thực hiện cài đặt các thư viện cần thiết

In [ ]:
!pip install flwr tensorflow numpy pandas scikit-learn
!pip install -q flwr[simulation] tensorflow
!pip install ray[rllib]  # also recommended: ray[debug]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.3/47.3 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.68.1
    Uninstalling grpcio-1.68.1:
      Successfully uninstalled grpcio-1.68.1
  Attempting uninstall: cryptography
    Found existing installation: cryptography 43.0.3
    Uninstalling cryptography-43.0.3:
      Successfully uninstalled cryptography-43.0.3
  Attempting uninstall: typer
    Found existing installation: typer 0.15.0
    Uninstalling typer-0.15.0:
      Successfully uninstalled typer-0.15.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 925

Bước 2: Import driver - Có thể không cần chạy

In [ ]:
# Connect Driver
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Bước 3: Import các thư viện

In [ ]:
# import libraries
import ray
from pyarrow import csv
import matplotlib.pyplot as plt
import seaborn as sns
import flwr as fl
from flwr.common import Metrics
from flwr.simulation.ray_transport.utils import enable_tf_gpu_growth
import math
from typing import Dict, List, Tuple
import glob
from flwr.simulation import run_simulation
import tensorflow as tf

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, precision_recall_curve, auc, average_precision_score, f1_score
from tensorflow.keras.models import load_model
import pandas as pd
import numpy as np
import sys
import sklearn
import io
import random
import matplotlib.pyplot as plt
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, GRU
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, Normalizer
from sklearn.model_selection import train_test_split


#Features - CIC_UNSW-NB15

Bước 4: Khai báo features có trong CIC-UNSWNB15

In [ ]:
CSV_HEADER=['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min', 'Label', 'attack_cat'] # bo Src_IP, Dst_IP, Timestamp, {'Flow ID', 'Src Port', 'Dst Port', 'Protocol'}
CSV_HEADER_WITHOUT_LABEL = ['Flow Duration', 'Total Fwd Packet', 'Total Bwd packets',
       'Total Length of Fwd Packet', 'Total Length of Bwd Packet',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Packet Length Min', 'Packet Length Max', 'Packet Length Mean',
       'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count',
       'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count',
       'URG Flag Count', 'CWR Flag Count', 'ECE Flag Count', 'Down/Up Ratio',
       'Average Packet Size', 'Fwd Segment Size Avg', 'Bwd Segment Size Avg',
       'Fwd Bytes/Bulk Avg', 'Fwd Packet/Bulk Avg', 'Fwd Bulk Rate Avg',
       'Bwd Bytes/Bulk Avg', 'Bwd Packet/Bulk Avg', 'Bwd Bulk Rate Avg',
       'Subflow Fwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Packets',
       'Subflow Bwd Bytes', 'FWD Init Win Bytes', 'Bwd Init Win Bytes',
       'Fwd Act Data Pkts', 'Fwd Seg Size Min', 'Active Mean', 'Active Std',
       'Active Max', 'Active Min', 'Idle Mean', 'Idle Std', 'Idle Max',
       'Idle Min']

# Bảng mã hóa
encoded_to_original = {
    0: 'Benign',
    1: 'Analysis',
    2: 'Backdoor',
    3: 'DoS',
    4: 'Exploits',
    5: 'Fuzzers',
    6: 'Generic',
    7: 'Reconnaissance',
    8: 'Shellcode',
    9: 'Worms'
}
# decoded_labels = [encoded_to_original[label] for label in y_pred]
# decoded_labels

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


#CIC_UNSW_NB15

Bước 6: Đọc và xử lý dữ liệu

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/AnToanHeThongThongTin-CH2024/CIC_UNSW-LABEL_Attack_cat.csv")
df = df.drop(['attack_cat'], axis=1) #Tách attack_cat ra khỏi
scaler = StandardScaler() # sử dụng Standard Scaler để xử lý dữ liệu.
scaler = scaler.fit(df[CSV_HEADER_WITHOUT_LABEL])
df[CSV_HEADER_WITHOUT_LABEL] = scaler.transform(df[CSV_HEADER_WITHOUT_LABEL])

In [ ]:
#Chia data_test
train_data,test_data = train_test_split(df,test_size=0.2,random_state=40)
train_data_1 ,train_data_2 = train_test_split(train_data,test_size=0.5,random_state=40)
train_data_1.to_csv("train_data_1.csv", index=False, header=True)
train_data_2.to_csv("train_data_2.csv", index=False, header=True)
#train_data.to_csv("train_data.csv", index=False, header=True)
test_data.to_csv("test_data.csv", index=False, header=True)

#Client and Server for Distributed

Bước 7: Khởi tạo FlowerClient

In [ ]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, x_train, y_train,model) -> None:
        # Create model
        self.model = model
        self.x_train, self.y_train = x_train, y_train

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):

        self.model.set_weights(parameters)
        self.model.fit(
             self.x_train,  self.y_train, epochs=3, batch_size=32, verbose=VERBOSE
        )
        return self.model.get_weights(), len(self.x_train), {}

In [ ]:
def get_client_fn(dataset_partitions,model):
    """Return a function to be executed by the VirtualClientEngine in order to construct
    a client."""

    def client_fn(cid: str) -> fl.client.Client:
        """Construct a FlowerClient with its own dataset partition."""

        # Extract partition for client with id = cid
        x_train, y_train = dataset_partitions[int(cid)]

        # Create and return client
        return FlowerClient(x_train, y_train, model)

    return client_fn


def partition_dataset(x, y, num_clients):
    partition_size = math.floor(len(x) / num_clients)
    partitions = []
    for i in range(num_clients):
        idx_from, idx_to = i * partition_size, (i + 1) * partition_size
        partitions.append((x[idx_from:idx_to], y[idx_from:idx_to]))
    return partitions

def get_evaluate_fn(model,file_path):
    """Return an evaluation function for server-side (i.e. centralized) evaluation."""

    # The `evaluate` function will be called after every round by the strategy
    def evaluate(
        server_round: int,
        parameters: fl.common.NDArrays,
        config: Dict[str, fl.common.Scalar],
    ):
        model.set_weights(parameters)  # Update model with the latest parameters
        model.save(file_path)
        # model_report(model,x_test,y_test)
        return

    return evaluate


## Chạy 1 trong hai bước sau

Bước 7.1 - Dành cho multiclass

In [ ]:
def get_partitions_multiclass(data_path):
    partitions = []

    for f in glob.glob(data_path):
        data = pd.read_csv(f, header=0)

        data.columns = data.columns.str.strip()

        # data = data[data["Class"] != ' null']
        # data["Class"] = data["Class"].replace({' "-1"' : 0 , ' "Facet"' : 1 })
        # data["Class"] = data["Class"].replace({'-1' : 0, 'Facet' : 1 })
        # data["Class"] = data["Class"].replace({' -1' : 0, ' Facet' : 1 })
        # data["Class"] = data["Class"].replace(np.nan,'benign')
        #data["label"] = np.where(data["label"]=='normal',0,1) # 'normal' -> 0, other ->1
        #data = data.sample(frac = 1).reset_index(drop=True)

        x_data = data.iloc[:,:-1]
        y_data = data["Label"]

        partitions.append((x_data.to_numpy(),y_data.to_numpy()))

    return partitions

Bước 7.2 - Dành cho Binary class

In [ ]:
def get_partitions_binary(data_path):
    partitions = []

    for f in glob.glob(data_path):
        data = pd.read_csv(f, header=0)

        data.columns = data.columns.str.strip()

        # data = data[data["Class"] != ' null']
        # data["Class"] = data["Class"].replace({' "-1"' : 0 , ' "Facet"' : 1 })
        # data["Class"] = data["Class"].replace({'-1' : 0, 'Facet' : 1 })
        # data["Class"] = data["Class"].replace({' -1' : 0, ' Facet' : 1 })
        # data["Class"] = data["Class"].replace(np.nan,'benign')
        data["Label"] = np.where(data["Label"]==0,0,1) # 'normal' -> 0, other ->1
        #data = data.sample(frac = 1).reset_index(drop=True)

        x_data = data.iloc[:,:-1]
        y_data = data["Label"]

        partitions.append((x_data.to_numpy(),y_data.to_numpy()))

    return partitions

#CREATE MODEL

Bước 8 - Hàm khởi tạo model - Chạy 1 trong hai tương ứng với Binary và Classification

### Binary Class

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.neural_network import MLPClassifier

# ANN Model Creationg Function
def create_ann_model_binary(input_shape=(76,), hidden_units=None):
    """
    Create a feedforward neural network model for binary classification with
    the specified input shape.

    Parameters:
    - input_shape: Tuple specifying the shape of the input data.
    - hidden_units: List of integers specifying the number of units in each hidden layer.
                    Default is [64, 32].

    Returns:
    - A compiled Keras model.
    """

    if hidden_units is None:
        hidden_units = [64, 32]

    model = Sequential()

    # Input layer
    model.add(Dense(hidden_units[0], activation='relu', input_shape=input_shape))
    model.add(Dropout(0.2))

    # Hidden layers
    for units in hidden_units[1:]:
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(0.2))

    # Output layer for binary classification
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='nadam', metrics=['accuracy'])
    return model

###Classification Class

In [ ]:
def create_ann_model_multiclass(input_shape=(76,), hidden_units=None, num_classes=10):
    """
    Create a feedforward neural network model for multiclass classification with
    the specified input shape and number of classes.

    Parameters:
    - input_shape: Tuple specifying the shape of the input data.
    - hidden_units: List of integers specifying the number of units in each hidden layer.
                    Default is [64, 32].
    - num_classes: Integer specifying the number of output classes (e.g., 10 for multiclass classification).

    Returns:
    - A compiled Keras model.
    """
    if hidden_units is None:
        hidden_units = [64, 32]

    model = Sequential()

    # Input layer
    model.add(Dense(hidden_units[0], activation='relu', input_shape=input_shape))
    model.add(Dropout(0.2))

    # Hidden layers
    for units in hidden_units[1:]:
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(0.2))

    # Output layer for multiclass classification
    model.add(Dense(num_classes, activation='softmax'))

    # Compile the model
    model.compile(
        loss='sparse_categorical_crossentropy',  # Loss function for integer labels
        optimizer='nadam',
        metrics=['accuracy']
    )

    return model

#Train and TEST_model

Bước 9: Phần Train Và Test sẽ chia làm hai phần - Train và Test dữ liệu - Federated Learning với ai loại là Binary và Classification

#Train model with Distributed Learning

### Bước 9 - Khai báo các Global setting


In [ ]:
VERBOSE = 0
NUM_CLIENTS = 2
# With a dictionary, you tell Flower's VirtualClientEngine that each
# client needs exclusive access to these many resources in order to run
client_resources = {"num_cpus":2,"num_gpus": 0}

### Train

Bước 10 - Train - Chọn một trong cái block sau tương ứng với Binary Và Classification

###Bước 10.1 - Binary training

In [ ]:
from flwr.simulation import run_simulation,start_simulation
result = []

partitions = get_partitions_binary("./dataset/*") #Cho Phân loại nhị phân
#partitions = get_partitions_multiclass("./dataset/*") #Cho Phân loại đa lớp

for _ in range(2):

    hidden_units = [64,32]
    output_model = f"./models/binary/i_{_}.keras" #Lưu trong Binary
    #output_model = f"./models/multiclass/i_{_}.keras" #Lưu trong Multiclass

    model = create_ann_model_binary(hidden_units=hidden_units) #làm cho nhị phân
    #model = create_ann_model_multiclass(hidden_units=hidden_units) #Làm cho Multiclass

    strategy = fl.server.strategy.FedAvg(
    fraction_fit=1,
    # min_fit_clients=10,
    fraction_evaluate = 1,
    min_available_clients = NUM_CLIENTS,
    evaluate_fn = get_evaluate_fn(model,output_model),
    )
#     strategy = MedianFedAvg(
#     fraction_fit=1,
#     fraction_evaluate=1,
#     min_available_clients=NUM_CLIENTS,
#     evaluate_fn=get_evaluate_fn(model, output_model),
# )

    # Start simulation
    history = fl.simulation.start_simulation(
    client_fn=get_client_fn(partitions,model),
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
    client_resources=client_resources,
    # actor_kwargs={
    # "on_actor_init_fn": enable_tf_gpu_growth  },
    )
    del history
    del strategy
    del model

    # model = load_model(output_model)
    # s = model_report_list(model,test_data.copy())

    # result.append(s)
    # del model


###Bước 10.2 - Train Classification

In [ ]:
from flwr.simulation import run_simulation,start_simulation
result = []

#partitions = get_partitions_binary("./dataset/*") #Cho Phân loại nhị phân
partitions = get_partitions_multiclass("./dataset/*") #Cho Phân loại đa lớp

for _ in range(2):

    hidden_units = [64,32]
    #output_model = f"./models/binary/i_{_}.keras" #Lưu trong Binary
    output_model = f"./models/multiclass/i_{_}.keras" #Lưu trong Multiclass

    #model = create_ann_model_binary(hidden_units=hidden_units) #làm cho nhị phân
    model = create_ann_model_multiclass(hidden_units=hidden_units) #Làm cho Multiclass

    strategy = fl.server.strategy.FedAvg(
    fraction_fit=1,
    # min_fit_clients=10,
    fraction_evaluate = 1,
    min_available_clients = NUM_CLIENTS,
    evaluate_fn = get_evaluate_fn(model,output_model),
    )
#     strategy = MedianFedAvg(
#     fraction_fit=1,
#     fraction_evaluate=1,
#     min_available_clients=NUM_CLIENTS,
#     evaluate_fn=get_evaluate_fn(model, output_model),
# )

    # Start simulation
    history = fl.simulation.start_simulation(
    client_fn=get_client_fn(partitions,model),
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=3),
    strategy=strategy,
    client_resources=client_resources,
    # actor_kwargs={
    # "on_actor_init_fn": enable_tf_gpu_growth  },
    )
    del history
    del strategy
    del model

    # model = load_model(output_model)
    # s = model_report_list(model,test_data.copy())

    # result.append(s)
    # del model


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
	Instead, use the `flwr run` CLI command to start a local simulation in your Flower app, as shown for example below:

		$ flwr new  # Create a new Flower app from a template

		$ flwr run  # Run the Flower app in Simulation Mode

	Using `start_simulation()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      Starting Flower simulation, config: num_rounds=3, no round_timeout
2024-12-09 03:54:14,390	INFO worker.py:1752 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'CPU': 2.0, 'memory': 7982530560.0, 'object_store_memory': 39

# Test model with Distributed Training

Bước 11 - Đọc dữ liệu test

In [ ]:
# Prepare Data for Testing - Vẫn con Test data còn lưu lại.
test_df= pd.read_csv("./test_data.csv")


## Thực hiện test cho model Nhị phân - Chạy từ trên xuống theo thứ tự.

In [ ]:
test_df["Label"] = np.where(test_df["Label"] == 0, 0, 1)
test_df["Label"].value_counts()

,count
Label,
0,71591
1,17992


In [ ]:
model_test = load_model('/content/models/binary/i_0.keras')
y_pred = model_test.predict(test_df.iloc[:,:-1])

In [ ]:
optimal_threshold = 0.5
y_pred_prob = (y_pred > optimal_threshold).astype("int32")
print(confusion_matrix(test_df["Label"],y_pred_prob))

In [ ]:
result = []
for _ in range(2):

    output_model = f"./models/binary/i_{_}.keras"
    print('--------------------Model thứ: ', _ )
    model = load_model(output_model)
    s = model_report_list(model, test_df.copy())
    #s = model_report_list_multiclass(model, test_data.copy())

    result.append(s)
    del model

--------------------Model thứ:  0


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 15 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


--------------------Model thứ:  1


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 15 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [ ]:
def model_report_list(model, df_test):
    result = []

    x_test = df_test.iloc[:, :-1]
    y_test = df_test["Label"]

    # Get model predictions for the test data
    y_pred_prob = model.predict(x_test, verbose=0)

    optimal_threshold = 0.5

    y_pred = (y_pred_prob > optimal_threshold).astype("int32")

    df_test["PLabel"] = y_pred

    # Calculate F1 Score
    f1 = f1_score(y_test, y_pred)
    result.append(('F1 Score', f1))

    # Calculate AUC
    auc_score = roc_auc_score(y_test, y_pred_prob)
    result.append(('AUC', auc_score))

    # Calculate Average Precision
    avg_precision = average_precision_score(y_test, y_pred_prob)
    result.append(('Average Precision', avg_precision))

    # Calculate Precision
    precision = precision_score(y_test, y_pred)
    result.append(('Precision', precision))

    # Calculate Recall
    recall = recall_score(y_test, y_pred)
    result.append(('Recall', recall))

    confusion_mat = confusion_matrix(y_test, y_pred)

    FP = confusion_mat[0][1]
    FN = confusion_mat[1][0]

    TP = confusion_mat[1][1]
    TN = confusion_mat[0][0]

    FPR = FP / (FP + TN) if (FP + TN) > 0 else 0  # Avoid division by zero
    FNR = FN / (FN + TP) if (FN + TP) > 0 else 0  # Avoid division by zero

    result.append(('False Positive Rate', FPR))
    result.append(('False Negative Rate', FNR))

    del model

    return result

In [ ]:
data = {}

# Iterate through each tuple in the first list to use as keys and create lists for their values
for metric, value in result[0]:
    data[metric] = []

# Fill the lists with values from each corresponding position in the remaining lists
for res in result:
    for i, (metric, value) in enumerate(res):
        data[metric].append(value)

# Convert the dictionary into a DataFrame
result_df = pd.DataFrame(data)

print('MEAN_AVG')
print("-"*100)
print(result_df.mean(axis=0))
print("-"*100)

MEAN_AVG
----------------------------------------------------------------------------------------------------
F1 Score               0.949063
AUC                    0.994243
Average Precision      0.969495
Precision              0.907971
Recall                 0.994108
False Positive Rate    0.025338
False Negative Rate    0.005892
dtype: float64
----------------------------------------------------------------------------------------------------


## Thực hiện test cho model Phân loại và xuất kết quả - chạy từ trên xuống

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def model_report_list_multiclass(model, df_test):
    result = []

    x_test = df_test.iloc[:, :-1]
    y_test = df_test["Label"]

    # Get model predictions for the test data
    y_pred_prob = model.predict(x_test, verbose=0)

    # Get predicted labels for multiclass
    y_pred = y_pred_prob.argmax(axis=1)

    df_test["PLabel"] = y_pred

    mapped_y_pred = [encoded_to_original[label] for label in y_pred]
    mapped_y_test = [encoded_to_original[label] for label in y_test]

    # Generate classification report
    class_report = classification_report(mapped_y_test, mapped_y_pred, output_dict=True)
    #print(class_report)
    result.append(('Classification Report', class_report))

    # # Calculate confusion matrix
    # confusion_mat = confusion_matrix(y_test, y_pred)
    # result.append(('Confusion Matrix', confusion_mat))

    del model

    return result


Report-Multiclass

In [ ]:
result = []
for _ in range(2):

    output_model = f"./models/multiclass/i_{_}.keras"
    print('--------------------Model thứ: ', _ )
    model = load_model(output_model)
    #s = model_report_list(model, test_data.copy())
    s = model_report_list_multiclass(model, test_data.copy())

    result.append(s)
    del model

--------------------Model thứ:  0


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 15 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMet

--------------------Model thứ:  1


/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 15 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMet

Chạy lệnh dưới để in ra thông tin các thông số tương ứng với từng Class trong phân loại tấn công.

In [ ]:
import pandas as pd

# Tạo một DataFrame để lưu các giá trị trung bình
df_avg_classes = pd.DataFrame(columns=['precision', 'recall', 'f1-score', 'support', 'accuracy'])

# Lấy các class từ báo cáo đầu tiên
classes = list(result[0][0][1].keys())  # result[0][0][1] chứa dictionary của classification report

# Xóa các chỉ số tổng hợp (macro avg, weighted avg, accuracy, support)
classes = [cls for cls in classes if cls not in ['accuracy', 'macro avg', 'weighted avg']]

# Tính trung bình cho từng class và lấy support từ báo cáo đầu tiên
for cls in classes:
    precision_avg = 0
    recall_avg = 0
    f1_avg = 0
    # Lặp qua các báo cáo trong result để tính trung bình precision, recall, f1-score
    for report in result:
        precision_avg += report[0][1][cls]['precision']
        recall_avg += report[0][1][cls]['recall']
        f1_avg += report[0][1][cls]['f1-score']

    # Tính trung bình
    num_reports = len(result)

    # Lấy support từ báo cáo đầu tiên
    support = result[0][0][1][cls]['support']

    df_avg_classes.loc[cls] = [
        precision_avg / num_reports,
        recall_avg / num_reports,
        f1_avg / num_reports,
        support,  # Thêm support vào DataFrame
        None  # Sẽ tính accuracy sau
    ]

# Tính trung bình cho các chỉ số tổng hợp (macro avg và weighted avg)
for avg_type in ['macro avg', 'weighted avg']:
    precision_avg = 0
    recall_avg = 0
    f1_avg = 0
    for report in result:
        precision_avg += report[0][1][avg_type]['precision']
        recall_avg += report[0][1][avg_type]['recall']
        f1_avg += report[0][1][avg_type]['f1-score']

    # Sử dụng support từ 'macro avg' trong báo cáo đầu tiên
    support = result[0][0][1][avg_type]['support'] if avg_type in result[0][0][1] and 'support' in result[0][0][1][avg_type] else None

    df_avg_classes.loc[avg_type] = [
        precision_avg / num_reports,
        recall_avg / num_reports,
        f1_avg / num_reports,
        support,  # Thêm support nếu có
        None  # Sẽ tính accuracy sau
    ]

# Tính trung bình accuracy cho tất cả các mô hình
accuracy_avg = 0
for report in result:
    accuracy_avg += report[0][1]['accuracy']

df_avg_classes['accuracy'] = accuracy_avg / num_reports

# In kết quả trung bình
print(df_avg_classes)

                precision    recall  f1-score  support  accuracy
Analysis         0.000000  0.000000  0.000000     75.0  0.913276
Backdoor         0.000000  0.000000  0.000000     90.0  0.913276
Benign           0.999448  0.973537  0.986322  71591.0  0.913276
DoS              0.928571  0.008037  0.015935    871.0  0.913276
Exploits         0.742485  0.664342  0.701178   6237.0  0.913276
Fuzzers          0.494579  0.933755  0.646649   6008.0  0.913276
Generic          0.846335  0.326206  0.470885    912.0  0.913276
Reconnaissance   0.804409  0.616248  0.697389   3342.0  0.913276
Shellcode        0.000000  0.000000  0.000000    413.0  0.913276
Worms            0.000000  0.000000  0.000000     44.0  0.913276
macro avg        0.481583  0.352212  0.351836  89583.0  0.913276
weighted avg     0.931234  0.913276  0.911380  89583.0  0.913276
